# Pivot - Unpivot

## Nhóm câu lệnh cơ bản


```sql
-- Tạo bảng
create table #student
(
id int,
mark1 float,
mark2 float,
mark3 float
)

insert into #student values (1, 1,2,3)
insert into #student values (2, 5,6,7)
insert into #student values (3, 7,4,2)

-- UNPIVOT

SELECT * FROM
(SELECT *
FROM #Student) a
UNPIVOT
(value_var FOR group_var IN (Mark1, Mark2, Mark3)
) b

-- PIVOT

SELECT * 
into #student_pivot FROM
(SELECT *
FROM #Student) a
UNPIVOT
(value_var FOR group_var IN (Mark1, Mark2, Mark3)
) b


select * from 
(select * from #student_pivot) a
pivot 
(sum(value_var) for group_var in (mark1, mark2, mark3)) b

```

## Procedure pivot

- Dynamic sql


```sql
declare @data nvarchar(30)
declare @group nvarchar(30)
set @data = '#student_pivot'
set @group = STUFF(
(SELECT ',' + b.group_var
              FROM 
			  (select distinct group_var from #student_pivot
			  where group_var not like 'mark1') b
              FOR XML PATH ('')),1,1,'')

declare @sqlstr nvarchar(max)
set @sqlstr = N'select * from 
(select * from '+ @data +') a
pivot 
(sum(value_var) for group_var in (' + @group + ')) b
'
exec sp_executesql @sqlstr

```

- Procedure



```sql
drop procedure if exists sql_pivot
create procedure sql_pivot  @data nvarchar(max),
							@group nvarchar(max), -- Variable to pivot
							@row nvarchar(max), -- Column to keep as key
							@value nvarchar(max), -- Column to calculate
							@function nvarchar(max) = 'sum', -- Function to calculate
							@stored_data nvarchar(max) = '##pivot_result' -- Store data result
as
begin
declare @sql nvarchar(max)
set @sql = '
declare @group_value nvarchar(max)
set @group_value = STUFF(
(SELECT '','' + b.'+@group+'
              FROM 
			  (select distinct '+ @group +' from '+ @data +') b
              FOR XML PATH ('''')),1,1,'''')

declare @sqlstr nvarchar(max)
set @sqlstr = N''
drop table if exists '+ @stored_data +'
select * 
into '+ @stored_data +'
from 
(select '+ @group+ ',' + @row + ', '+@value +'  from '+ @data +') a
pivot 
('+@function+'('+ @value +') for '+@group+' in ('' + @group_value + '')) b
''
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end;

```


- Ví dụ:


```sql
sql_pivot @data = '##student_pivot', @row = 'id',
@group= 'group_var', @value = 'value_var', @function = 'sum'
select * from ##pivot_result

```

## Procedure unpivot

- Dynamic sql: Cho phép nhiều ID

- Cách 1: Chỉ sử dụng với các data trong máy local


```sql
declare @data nvarchar(max) = 'student'
declare @id_var nvarchar(max) = '''id'',''mark1'''
declare @group_var nvarchar(max) = 'group_var'
declare @value_var nvarchar(max) = 'value_var'

declare @sql nvarchar(max)
set @sql = '
declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(column_name) 
                    FROM (
                        select   table_name, column_name, ordinal_position, data_type
                        from   information_schema.columns
                        where   table_name = '''+@data+''' and 
						column_name not in ('+@id_var+') 
                        ) cols
                    ORDER BY ordinal_position               
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @cols

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(column_name) 
                    FROM (
                        select   table_name, column_name, ordinal_position, data_type
                        from   information_schema.columns
                        where   table_name = '''+@data+''' and 
						column_name in ('+@id_var+') 
                        ) cols
                    ORDER BY ordinal_position               
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @key

declare @sqlstr nvarchar(max)
set @sqlstr = N''
            select ''+@key+'', '+@group_var+', '+@value_var+'
            from(
               select *
               from '+@data+'
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b''
print @sqlstr
exec sp_executesql @sqlstr
'
  
exec sp_executesql @sql

```

- Cách 2: Khái quát dynamic sql



```sql
declare @db nvarchar(max) = 'tempdb'
declare @data nvarchar(max) = '##student'
declare @id_var nvarchar(max) = '''id'''
declare @group_var nvarchar(max) = 'group_var'
declare @value_var nvarchar(max) = 'value_var'
declare @stored_data nvarchar(max) = '##a'
declare @sql nvarchar(max) 

set @sql = N'
drop table if exists ##all_cols
select name into ##all_cols
from '+@db+'.sys.columns 
where object_id in (select object_id from '+@db+'.sys.tables where name like ''%'+@data+''')

declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name not in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @cols

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
select @key

--Run procedure

declare @sqlstr nvarchar(max)
set @sqlstr = N''
			drop table if exists '+@stored_data+'
            select ''+@key+'', '+@group_var+', '+@value_var+'
			into '+@stored_data+'
            from(
               select *
               from '+@db+'.dbo.['+@data+']
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b
			select * from '+@stored_data+'
			''
print @sqlstr
exec sp_executesql @sqlstr
'
exec sp_executesql @sql

```

**Lưu ý**: Các bảng phải là bảng trong db hoặc bảng lưu ở global environment

- Procedure


```sql
drop procedure if exists sql_unpivot
create procedure sql_unpivot 
 @db nvarchar(max) = 'tempdb', --Database contains data
@data nvarchar(max), -- Name of data
@id_var nvarchar(max), -- Key variables
@group_var nvarchar(max) = 'group_var', --Name of groups variable
@value_var nvarchar(max) = 'value_var', -- Name of value variable
@stored_data nvarchar(max) = '##unpivot_result'  -- Data storage
as 
begin
declare @sql nvarchar(max) 
set @sql = N'
drop table if exists ##all_cols
select name into ##all_cols
from '+@db+'.sys.columns 
where object_id in (select object_id from '+@db+'.sys.tables where name like ''%'+@data+''')

declare @cols AS NVARCHAR(MAX)
   select @cols = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name not in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show columns
print @cols 

declare @key AS NVARCHAR(MAX)
   select @key = STUFF((SELECT '','' + QUOTENAME(name) 
                    FROM (
                        select name from ##all_cols 
                        where name in ('+@id_var+') 
                        ) cols            
                    FOR XML PATH(''''), TYPE).value(''.'', ''NVARCHAR(MAX)''),1,1,'''')
-- Show key
print @key 

--Run procedure

declare @sqlstr nvarchar(max)
set @sqlstr = N''
			drop table if exists '+@stored_data+'
            select ''+@key+'', '+@group_var+', '+@value_var+'
			into '+@stored_data+'
            from(
               select *
               from '+@db+'.dbo.['+@data+']
            ) as a
            unpivot
            (
              '+@value_var+' for '+@group_var+' in ('' + @cols + '')
            ) as b
			select * from '+@stored_data+'
			''
print @sqlstr
exec sp_executesql @sqlstr
'
exec sp_executesql @sql
end

```

```sql
exec sql_unpivot @db = 'loan_db', @data = 'MONTHLY_LCY_AMOUNT - M74 - BICDATA', 
@id_var = '''id'', ''YEARMONTH'', ''APP'',''ACCTNO'', ''CIF'', ''AVERAGE_BAL'', ''BI_SEGMENT'''

```

**Lưu ý**:

- Bắt buộc phải có alias sau bảng


# Chia một cột thành nhiều cột


```sql
CREATE TABLE tbl_SplitColumns (col1 varchar(max))
GO
 
INSERT INTO tbl_SplitColumns 
VALUES ('part1 part2 part3'),('abc xyz pqr'),('one two three'), ('Hello man nua nha')
GO


select * from tbl_SplitColumns

DECLARE @delimiter VARCHAR(5)
SET @delimiter=' '  
;WITH CTE AS
( 
    SELECT
        col1,
        CAST('<A>' + REPLACE(col1, @delimiter , '</A><A>') + '</A>' AS XML) AS XMLString
    FROM tbl_SplitColumns 
)
select * from CTE
SELECT
     XMLString.value('/A[1]', 'varchar(10)') As C1
     ,XMLString.value('/A[2]', 'varchar(10)') As C2
     ,XMLString.value('/A[3]', 'varchar(10)') As C3
	 ,XMLString.value('/A[4]', 'varchar(10)') As C4
  FROM CTE 
GO

```